In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import glob
!pip install openpyxl

day_folder = "/content/drive/My Drive/11 april/"
file_paths = glob.glob(day_folder + "*.csv")

school_id = 42
class_id = 102

dataframes = [pd.read_csv(file, sep=';') for file in file_paths]
keyfile_general = pd.read_csv("Keyfile_csv.csv")
keyfile_specific = pd.read_excel("keyfile school 42 class 102.xlsx")


file_to_subject = {}
teacher_tag = '0x24046130BA41'

for file in file_paths:
    df_temp = pd.read_csv(file, sep=';')
    tag_id = df_temp['TagId'].iloc[0]  # This is the TrackLabID
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    try:
        subject_id = keyfile_specific.loc[keyfile_specific['tagnr'] == tagnr, 'subject ID'].values[0]
    except:
        print("Tag not found:" + tag_id)
        subject_id = None

    # Store TrackLabID (tag_id), Tagnumber, and subject ID in the dictionary
    file_to_subject[tag_id] = {'Tagnumber': tagnr, 'subject ID': subject_id}
#print(file_to_subject)

# Filter dataframes to remove unfound tags (except teacher)
filtered_dataframes = [
    df for df in dataframes
    if (file_to_subject.get(df['TagId'].iloc[0]) and
        file_to_subject[df['TagId'].iloc[0]]['subject ID'] is not None) or
       df['TagId'].iloc[0] == teacher_tag
]

dataframes = filtered_dataframes  # Replace original dataframes with filtered ones


Mounted at /content/drive
Tag not found:0x24025F44CDE1
Tag not found:0x24046130BA41
Tag not found:0x24046131F1EE


In [2]:


def process_timestamps(df):
    # Extract 'TrackLabID'
    tag_id = df['TagId'].iloc[0]

    try:
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], format='%d/%m/%Y %H:%M:%S.%f').dt.floor('s')
    except ValueError:
        # Handle cases where 'TimeStamp' is already in datetime format
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp']).dt.floor('s')

    # Keep only the first entry for each second
    df = df.groupby('TimeStamp').first().reset_index()

    for column in ['X', 'Y', 'Z']:
        # Iteratively detects and replaces outliers until no more are found
        while True:
            # Calculates the difference between consecutive values
            diff = df[column].diff()

            # Identify outliers where the difference is greater than 1.5
            outlier_indices = diff[abs(diff) > 1.5].index

            # loop break condition
            if len(outlier_indices) == 0:
                break

            # Replace outliers with the prior value
            for index in outlier_indices:
                if index > 0:  # >0 avoids first row (which has no prior value)
                    df.loc[index, column] = df.loc[index - 1, column]

    # Convert 'TimeStamp' to datetime in all dataframes
    for df_ in dataframes:
        df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%d/%m/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime

    time_range = pd.date_range(
        start=min(df_['TimeStamp'].min() for df_ in dataframes),
        end=max(df_['TimeStamp'].max() for df_ in dataframes),
        freq='s'
    )
    full_df = pd.DataFrame({'TimeStamp': time_range})

    # fills missing seconds
    df = pd.merge(full_df, df, left_on='TimeStamp', right_on='TimeStamp', how='left')

    # Forward fill and then backward fill to handle NaNs at the beginning and end
    df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']].ffill().bfill()
    # Interpolate any remaining NaNs
    df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']].interpolate(method='linear')

    # Add 'TagID' column back to the DataFrame
    df['TagId'] = tag_id
    return df

dataframes = [df for df in [process_timestamps(df) for df in dataframes] if df is not None]



<ipython-input-2-e959d0b037c3>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%d/%m/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime
<ipython-input-2-e959d0b037c3>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%d/%m/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime
<ipython-input-2-e959d0b037c3>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['

In [3]:
import pandas as pd

def find_extremes_and_context(df, column):
    """
    Manual check as outlier detection, was used to determine 1.5 as sensible value for allowed value jumps.
    """

    # Find index of minimum and maximum values
    min_index = df[column].idxmin()
    max_index = df[column].idxmax()

    # Print +-5 values around min/max
    print(f"Minimum value of {column}:")
    print(df.loc[max(0, min_index - 5):min(len(df), min_index + 5), ['TimeStamp', 'X', 'Y', 'Z']])

    print(f"\nMaximum value of {column}:")
    print(df.loc[max(0, max_index - 5):min(len(df), max_index + 5), ['TimeStamp', 'X', 'Y', 'Z']])


for df in dataframes:
    for column in ['X', 'Y']:
        find_extremes_and_context(df, column)

Minimum value of X:
                TimeStamp         X         Y         Z
16957 2023-11-04 14:00:05  4.062650  2.931207  0.494862
16958 2023-11-04 14:00:06  3.343875  2.931207  0.489167
16959 2023-11-04 14:00:07  3.012866  2.931207  0.690387
16960 2023-11-04 14:00:08  1.773875  2.931207  0.480015
16961 2023-11-04 14:00:09  0.911313  2.931207  0.449632
16962 2023-11-04 14:00:10  0.453502  2.931207  0.539180
16963 2023-11-04 14:00:11  0.453502  2.931207  0.385041
16964 2023-11-04 14:00:12  0.453502  2.931207  0.640938
16965 2023-11-04 14:00:13  1.472175  2.931207  0.501765
16966 2023-11-04 14:00:14  1.979949  2.931207  0.612660
16967 2023-11-04 14:00:15  2.287534  2.931207  0.237622

Maximum value of X:
                TimeStamp         X         Y         Z
16830 2023-11-04 13:57:58  5.828368  2.931207  0.536138
16831 2023-11-04 13:57:59  5.828368  2.931207  0.536138
16832 2023-11-04 13:58:00  5.828368  2.931207  0.536138
16833 2023-11-04 13:58:01  5.828368  2.931207  0.536138
16834 2

In [4]:
#don't actually use the averages, could be used for plotting though (I removed those cells from the notebook because we don't actually need it, could add back if you want)
object_averages = {}
teacher_tag = '0x24046130BA41'
for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    if tag_id == teacher_tag:
        tagnr = '35'
        subject_id = "Teacher"

    elif tag_id in df['TrackLabID'].values:
        tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
        subject_id = keyfile_specific.loc[keyfile_specific['tagnr'] == tagnr, 'subject ID'].values[0]




    avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].mean()
    object_averages[f"Person_{subject_id} with Tagnr {tagnr}"] = (avg_x, avg_y, avg_z)

avg_df = pd.DataFrame.from_dict(object_averages, orient="index", columns=["X", "Y", "Z"])
print(avg_df)

                                     X         Y         Z
Person_7.0 with Tagnr 7       4.555622  3.556851  0.478047
Person_10.0 with Tagnr 11     4.522219  1.256925  0.498803
Person_8.0 with Tagnr 9       4.513448  3.075386  0.364112
Person_Teacher with Tagnr 35  1.112946  5.289362  0.577825
Person_14.0 with Tagnr 18     4.939421  1.984068  0.424608
Person_15.0 with Tagnr 17     1.909339  3.802204  0.362416
Person_9.0 with Tagnr 10      3.612095  4.262204  0.488251
Person_6.0 with Tagnr 6       2.993484  1.904924  0.361871
Person_3.0 with Tagnr 2       4.000252  2.963590  0.448872
Person_4.0 with Tagnr 3       2.249096  3.909146  0.446787
Person_1.0 with Tagnr 1       1.780252  3.994702  0.427942
Person_11.0 with Tagnr 13     0.464603  1.753425  0.544174
Person_5.0 with Tagnr 5       3.640602  5.264295  0.508988
Person_13.0 with Tagnr 15     3.555486  2.939261  0.407455


In [5]:
fivemin_object_averages = {}
for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # Calculate average for the first 300 rows (= 5 minutes)
    avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].head(300).mean()
    fivemin_object_averages[f"{tagnr}"] = (avg_x, avg_y, avg_z)

avg_df = pd.DataFrame.from_dict(fivemin_object_averages, orient="index", columns=["X", "Y", "Z"])

print(avg_df)

           X         Y         Z
7   4.343513  3.896228  0.463156
11  4.795933  2.059340  0.584552
9   5.180991  3.668428  0.403377
35  3.224422  3.151180  0.572984
18  3.319264  1.217425  0.248889
17  3.015941  1.991201  0.314632
10  3.601501  4.338986  0.505315
6   5.655297  3.770819  0.708541
2   5.304984  3.610548  0.476911
3   3.286719  4.315914  0.295697
1   1.594196  4.053265  0.406329
13  0.972201  4.050405  0.565632
5   4.127128  5.090920  0.520120
15  1.646461  2.919265  0.485125


In [6]:
#this initially was for baseline distance purposes, decided using avg first 5 minutes might be better though due to trackers potentially not being turned on in usual positions (e.g. turning it on whilst picking it up from teachers desk)
object_start = {}

for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # respective 1st coordinates
    start_x = df_['X'].iloc[0]
    start_y = df_['Y'].iloc[0]
    start_z = df_['Z'].iloc[0]
    object_start[f"{tagnr}"] = (start_x, start_y, start_z)

start_positions_df = pd.DataFrame.from_dict(object_start, orient="index", columns=["X", "Y", "Z"])

print(start_positions_df)

           X         Y         Z
7   4.378336  3.858561  0.519511
11  4.137252  1.136436  0.547842
9   5.180991  3.668428  0.403377
35  2.322620  5.061799  0.677834
18  3.228317  1.148220  0.224174
17  3.007175  2.026214  0.504837
10  3.570728  4.738615  0.631311
6   5.477901  3.812096  0.405335
2   5.304984  3.610548  0.408014
3   4.663750  4.402218  0.126461
1   1.834673  4.049643  0.235754
13  1.232346  4.027922  0.643666
5   4.257247  5.243555  0.548409
15  1.594550  2.928901  0.427727


In [7]:
import numpy as np

def euclidean_distance(x1, y1, x2, y2):
  return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

target_tagnr = 35

# Find target coordinates using tagnr
target_coords = None
for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker == f"{target_tagnr}":
    target_coords = (x, y)
    break




target_x, target_y = target_coords
min_distance = float('inf')
closest_tracker = None

for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker != f"{target_tagnr}":  # Excludes teacher who would have distance 0
    distance = euclidean_distance(target_x, target_y, x, y)
    print(distance, tracker) #can be removed later
    if distance < min_distance:
      min_distance = distance
      closest_tracker = tracker

closest_distance_to_35 = min_distance
print(f"The closest tracker to the teacher is {closest_tracker} with a distance of {closest_distance_to_35}")

1.3444183198351818 7
1.9135730285799242 11
2.0237860774653895 9
1.9360793412556296 18
1.1785655702505498 17
1.2462229545482348 10
2.508606713010092 6
2.1306703665284306 2
1.1663987552958601 3
1.8631676348317772 1
2.425098405877602 13
2.139501879208006 5
1.5949121695792414 15
The closest tracker to the teacher is 3 with a distance of 1.1663987552958601


In [8]:
tracker_counts = {}
total_teacher_proximity_count = 0
teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher

for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Initialise count for tracker
    tracker_counts[f"{tagnr}"] = 0


    # Iterate through rows using the index
    for index in range(len(df)):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < closest_distance_to_35:
            tracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
sorted_tracker_counts = dict(sorted(tracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within full distance of anyone: {total_teacher_proximity_count}")


exp_amount = 100 / (len(dataframes)-1) #determines expected proximity percentage if it were fully equal(e.g. 20 students = 5%), -1 due to not counting teacher
lower = 0.75
upper = 1.25
for tracker, count in sorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within full distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within full distance of anyone: 10433
Person 1.0 with Tracker 1: 3796 times within full distance of teacher, category: above average with proximity percentage 36.38
Person 4.0 with Tracker 3: 1925 times within full distance of teacher, category: above average with proximity percentage 18.45
Person 5.0 with Tracker 5: 1421 times within full distance of teacher, category: above average with proximity percentage 13.62
Person 9.0 with Tracker 10: 558 times within full distance of teacher, category: below average with proximity percentage 5.35
Person 15.0 with Tracker 17: 549 times within full distance of teacher, category: below average with proximity percentage 5.26
Person 11.0 with Tracker 13: 536 times within full distance of teacher, category: below average with proximity percentage 5.14
Person 13.0 with Tracker 15: 438 times within full distance of teacher, category: below average with proximity percentage 4.20
Person 14.0 with Tracker 18: 387 times within full

In [9]:
half_distance = closest_distance_to_35 / 2
htracker_counts = {}
total_teacher_proximity_count = 0

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    htracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < half_distance:
            htracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
hsorted_tracker_counts = dict(sorted(htracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within half distance of anyone: {total_teacher_proximity_count}")
# Print the sorted counts with subject IDs


for tracker, count in hsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within half distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within half distance of anyone: 1033
Person 4.0 with Tracker 3: 264 times within half distance of teacher, category: above average with proximity percentage 25.56
Person 15.0 with Tracker 17: 142 times within half distance of teacher, category: above average with proximity percentage 13.75
Person 11.0 with Tracker 13: 111 times within half distance of teacher, category: above average with proximity percentage 10.75
Person 3.0 with Tracker 2: 90 times within half distance of teacher, category: average with proximity percentage 8.71
Person 8.0 with Tracker 9: 83 times within half distance of teacher, category: average with proximity percentage 8.03
Person 10.0 with Tracker 11: 74 times within half distance of teacher, category: average with proximity percentage 7.16
Person 6.0 with Tracker 6: 69 times within half distance of teacher, category: average with proximity percentage 6.68
Person 1.0 with Tracker 1: 69 times within half distance of teacher, category: aver

In [10]:
quarter_distance = closest_distance_to_35/4

qtracker_counts = {}
total_teacher_proximity_count = 0

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    qtracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < quarter_distance:
            qtracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
qsorted_tracker_counts = dict(sorted(qtracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within quarter distance of anyone: {total_teacher_proximity_count}")
# Print the sorted counts with subject IDs


for tracker, count in qsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within quarter distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within quarter distance of anyone: 170
Person 3.0 with Tracker 2: 70 times within quarter distance of teacher, category: above average with proximity percentage 41.18
Person 15.0 with Tracker 17: 30 times within quarter distance of teacher, category: above average with proximity percentage 17.65
Person 4.0 with Tracker 3: 26 times within quarter distance of teacher, category: above average with proximity percentage 15.29
Person 1.0 with Tracker 1: 12 times within quarter distance of teacher, category: average with proximity percentage 7.06
Person 8.0 with Tracker 9: 11 times within quarter distance of teacher, category: average with proximity percentage 6.47
Person 11.0 with Tracker 13: 6 times within quarter distance of teacher, category: below average with proximity percentage 3.53
Person 13.0 with Tracker 15: 6 times within quarter distance of teacher, category: below average with proximity percentage 3.53
Person 6.0 with Tracker 6: 5 times within quarter dis

In [17]:
import pandas as pd


merge_df = pd.read_excel("merged-data-WIP_2025-05-01_17-31.xlsx")  # Replace with your file name

# Create new columns if they don't exist
for distance_type in ['full', 'half', 'quarter']:
    if f'{distance_type}_proximity' not in merge_df.columns:
        merge_df[f'{distance_type}_proximity'] = None
    if f'{distance_type}_category' not in merge_df.columns:
        merge_df[f'{distance_type}_category'] = None


for distance_type, sorted_counts in [('full', sorted_tracker_counts), ('half', hsorted_tracker_counts), ('quarter', qsorted_tracker_counts)]:
    total_count = sum(sorted_counts.values()) # total proximity count for this distance type

    for tracker, count in sorted_counts.items():

        tag_id = df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0]

        # Find the matching row in the existing DataFrame
        mask = (merge_df['school'] == school_id) & (merge_df['class'] == class_id) & (merge_df['tracklab_id'] == tag_id)  # Replace 'tracklab_id' with the actual column name for TagID if different
        matching_row_index = merge_df[mask].index

        # Update the existing DataFrame if matching row is found
        if len(matching_row_index) > 0:
            row_index = matching_row_index[0]
            merge_df.loc[row_index, f'{distance_type}_proximity'] = (count / total_count) * 100

            proximity_percentage = (count / total_count) * 100
            ratio = proximity_percentage / exp_amount

            if ratio < lower:
                category = "below average"
            elif ratio > upper:
                category = "above average"
            else:
                category = "average"

            merge_df.loc[row_index, f'{distance_type}_category'] = category


merge_df.to_excel("merged-data-WIP_2025-05-01_17-31.xlsx", index=False)

0x24025F48A3E6
0x24025F44F8D7
0x24046131F437
0x24046130C9C5
0x24046131F062
0x24046131EF0E
0x24046131F584
0x24025F44E6FB
0x24025F48A133
0x24025F44F682
0x24025F465724
0x24046130C8AB
0x24046130BB1E
0x24025F44F8D7
0x24046131F062
0x24046131EF0E
0x24025F48A133
0x24046130C8AB
0x24025F44F682
0x24046130BB1E
0x24025F48A3E6
0x24046130C9C5
0x24046131F584
0x24046131F437
0x24025F465724
0x24025F44E6FB
0x24025F48A133
0x24046131F062
0x24025F44F8D7
0x24025F48A3E6
0x24046130C8AB
0x24046131EF0E
0x24046131F584
0x24046130BB1E
0x24025F44F682
0x24046130C9C5
0x24025F465724
0x24025F44E6FB
0x24046131F437
